A `Caller` is the basic structure that wraps all logic required for LLM call-and-response.

A `Caller` associates a `ConversationTemplate` with a specific LLM client and call parameters (assumes OpenAI-compatibility through a framework like `aisuite`).
This allows every Caller instance to use a different model and/or parameters, and sets expectations for the Caller instance.
Whereas `MessageTemplates` validate _inputs_ to the template and `Handlers` validate the LLM responses, `Callers` make it all happen.

Additionally, `Callers` can be used as functions/tools in tool-calling workflows by leveraging `Caller.signature()` which provides the inputs the `Caller.conversation_spec` requires as a JSON schema.
Since a `Caller` has a specific client and model assigned, this effectively allows us to use Callers to route to specific models for specific use cases.
Since Callers can behave as functions themselves, we enable complex workflows where Callers can call Callers (ad infinitum ad nauseum).

Simple factory functions create Callers where the use case is defined by their handlers:

- `ChatCaller`: a simple Caller implementation designed for chat messages without response validation.
- `RegexCaller`: uses regex for response validation.
- `StructuredCaller`: is intended for structured responses, and uses Pydantic for response validation.
- `ToolCaller`: a configuration for tool-use; can optionally invoke the tool based on arguments in the LLM's response and return the function results.Simple factory functions create Callers where the use case is defined by their handlers:


In [ ]:
import json
import logging
import os
import re
import textwrap
from typing import cast

import json_repair
from pydantic import BaseModel, Field, ValidationError as PydanticValidationError, create_model

import aisuite
import openai

from yaaal.core.caller import Caller, create_chat_caller, create_structured_caller, create_tool_caller
from yaaal.core.handler import ResponseHandler, ToolHandler
from yaaal.core.template import (
    ConversationTemplate,
    JinjaMessageTemplate,
    StaticMessageTemplate,
    StringMessageTemplate,
    UserMessageTemplate,
)
from yaaal.core.validator import PassthroughValidator, PydanticValidator, RegexValidator, ToolValidator
from yaaal.types_.base import JSON
from yaaal.types_.core import Conversation, Message
from yaaal.utilities import basic_log_config, format_json

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
basic_log_config()
logging.getLogger("yaaal").setLevel(logging.DEBUG)
logger = logging.getLogger(__name__).setLevel(logging.DEBUG)

## Quick Start


In [ ]:
# all Callers require a client and a model to call.
# `yaaal` is built around OpenAI-compatible APIs primarily provided by `aisuite`
client = aisuite.Client(
    provider_configs={
        "openai": {"api_key": os.environ["YAAAL_OPENAI_API_KEY"]},
        "anthropic": {"api_key": os.environ["YAAAL_ANTHROPIC_API_KEY"]},
        # ...
    }
)
# `aisuite` specifies models in "provider:model" format
# model = "openai:gpt-4o-mini"
model = "anthropic:claude-3-5-haiku-latest"

In [ ]:
# A `ChatCaller`
caller = create_chat_caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    conversation_template=ConversationTemplate(
        name="chat",
        description="A simple chat",
        conversation_spec=[
            Message(
                role="system",
                content="You are a helpful assistant",
            ),
            UserMessageTemplate(),
        ],
    ),
)

2025-03-06 20:31:02,088 - DEBUG    - yaaal.core.caller - request_params:110 - All API requests for Caller will use params : {'temperature': 0.7}


A Caller's call to the LLM is determined by the templates available in the ConversationTemplate and the ConversationSpec specification used to render the output.

A `ConversationTemplate` is a way to use various MessageTemplates to render a `Conversation`.
ConversationTemplates render the Conversation based on a conversation_spec, a sequence of templates/messages defining the conversation
You can provide `ConversationTemplate.render()` dictionary of variables used for rendering the message templates.
Each message template in the conversation specification is validated and rendered using these variables.


In [ ]:
# Caller `schema` attribute is based on its Prompt signature
print(format_json(caller.schema))

{
  "description": "A simple chat",
  "properties": {
    "user": {
      "title": "User",
      "type": "string",
    },
  },
  "required": [
    "user",
  ],
  "title": "chat",
  "type": "object",
}


In [ ]:
caller.conversation_template.render(
    {"user": "Who is Harry Potter?"},  # or user="Who is Harry Potter?"
)

{
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful assistant",
    },
    {
      "role": "user",
      "content": "Who is Harry Potter?",
    },
  ],
}

In [ ]:
# callers are called as functions to get the response from the LLM
response = caller(**{"user": "Who is Harry Potter?"})

print(textwrap.fill(response, replace_whitespace=False))

Harry Potter is a fictional character and the protagonist of the
popular book series written by British author J.K. Rowling. Here are
some key details about Harry Potter:

1. Background:
- An orphaned
wizard who survived an attack by the dark wizard Lord Voldemort when
he was a baby
- Raised by his non-magical (Muggle) aunt and uncle, the
Dursleys
- Discovers he is a wizard on his 11th birthday and attends
Hogwarts School of Witchcraft and Wizardry

2. Personal
characteristics:
- Known for his lightning bolt-shaped scar on his
forehead
- Brave, loyal, and often stands up against evil
- A talented
wizard, particularly skilled in Defense Against the Dark Arts
- Best
friends with Ron Weasley and Hermione Granger

3. Story arc:
- The
main storyline follows Harry's battles against Lord Voldemort
- Fights
to protect the wizarding world from dark forces
- Plays a crucial role
in defeating Voldemort in the final book

The Harry Potter series
consists of seven books that were published between 

In [ ]:
# A `RegexCaller` validates the response with a regex pattern
pattern = re.compile(r"\b[A-E]\b(?!.*\b[A-E]\b)")

system_instructions = """
The following are multiple choice questions (with answers) about Star Wars.

What is the model designation of an X-Wing?
A. T-65B
B. BTL-A4
C. RZ-1
D. A/SF-01
Answer: A
"""

user_template = """
{{question}}
Answer:
""".strip()


class MCQAQuestion(BaseModel):
    question: str = Field(description="The multiple choice question")


regex_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    conversation_template=ConversationTemplate(
        name="Star Wars QA",
        description="Multiple-choice question answering",
        conversation_spec=[
            StaticMessageTemplate(role="system", template=system_instructions),
            JinjaMessageTemplate(role="user", template=user_template, validation_model=MCQAQuestion),
        ],
    ),
    handler=ResponseHandler(validator=RegexValidator(pattern=pattern)),
)

2025-03-06 20:31:14,404 - WARNING  - yaaal.core.template - name:339 - Converted template name 'Star Wars QA' to 'star_wars_qa'
2025-03-06 20:31:14,405 - DEBUG    - yaaal.core.caller - request_params:110 - All API requests for Caller will use params : {'temperature': 0.7}


In [ ]:
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = regex_caller(**{"question": question})
print(response)

if response == "E":
    print("Success! 🎉")
# print(textwrap.fill(response, replace_whitespace=False))

E
Success! 🎉


In [ ]:
# A `StructuredCaller` validates the response with a Pydantic model, and is good for structure data extraction
class Person(BaseModel, extra="ignore"):
    """A Person's characteristics."""

    name: str
    age: int
    favorite_color: str


# Notes on fstrings with jinja templates --
# The fstring allows us to substitute in variables before
# dynamically rendering the template in the Prompt.
# Because we're using fstrings, we have to double the `{}`
# so python understands they do not indicate an fstring substitution.
system_instructions = f"""
Identify facts about a person as they introduce themselves.

Respond in a format that matches the following json schema:

<schema>
{Person.model_json_schema()}
</schema>
""".strip()

structured_caller = create_structured_caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    conversation_template=ConversationTemplate(
        name="person details",
        description="Identify details about a person",
        conversation_spec=[
            StaticMessageTemplate(role="system", template=system_instructions),
            UserMessageTemplate(),
        ],
    ),
    response_model=Person,
)

2025-03-06 20:31:16,248 - WARNING  - yaaal.core.template - name:339 - Converted template name 'person details' to 'person_details'
2025-03-06 20:31:16,250 - DEBUG    - yaaal.core.caller - request_params:110 - All API requests for Caller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'Person', 'strict': True, 'parameters': {'description': "A Person's characteristics.", 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'favorite_color': {'title': 'Favorite Color', 'type': 'string'}}, 'required': ['name', 'age', 'favorite_color'], 'title': 'Person', 'type': 'object', 'additionalProperties': False}, 'description': "A Person's characteristics."}}], 'tool_choice': {'type': 'tool', 'name': 'Person'}}


In [ ]:
introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

response = structured_caller(
    user=introduction,  # or **{"user": introduction}
)

print(type(response))
print(format_json(response.model_dump()))

<class '__main__.Person'>
{
  "name": "Bob",
  "age": 42,
  "favorite_color": "blue",
}


Callers using Pydantic Handlers still return an AssistantMessage; it was validated internally before returning to the user.

This means we still have to re-validate if we want the response as a Pydantic model.


In [ ]:
# A `ToolCaller` can choose to call tools or respond like a normal LLM.
tool_caller = create_tool_caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    conversation_template=ConversationTemplate(
        name="tool use",
        description="Determine which tool to use",
        conversation_spec=[
            Message(role="system", content="Use the best tool for the task."),
            UserMessageTemplate(),
        ],
    ),
    toolbox=[regex_caller, structured_caller],  # use other callers as tools!
    auto_invoke=True,  # make the recommended tool call
)

2025-03-06 20:31:21,879 - WARNING  - yaaal.core.template - name:339 - Converted template name 'tool use' to 'tool_use'
2025-03-06 20:31:21,881 - DEBUG    - yaaal.core.caller - request_params:110 - All API requests for Caller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'star_wars_qa', 'strict': True, 'parameters': {'description': 'Multiple-choice question answering', 'properties': {'question': {'description': 'The multiple choice question', 'title': 'Question', 'type': 'string'}}, 'required': ['question'], 'title': 'star_wars_qa', 'type': 'object', 'additionalProperties': False}, 'description': 'Multiple-choice question answering'}}, {'type': 'function', 'function': {'name': 'person_details', 'strict': True, 'parameters': {'description': 'Identify details about a person', 'properties': {'user': {'title': 'User', 'type': 'string'}}, 'required': ['user'], 'title': 'person_details', 'type': 'object', 'additionalProperties': False}, 'descriptio

In [ ]:
# the tool_caller will automatically add the tools to the request parameters
print(format_json(tool_caller.request_params))

{
  "temperature": 0.7,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "star_wars_qa",
        "strict": true,
        "parameters": {
          "description": "Multiple-choice question answering",
          "properties": {
            "question": {
              "description": "The multiple choice question",
              "title": "Question",
              "type": "string",
            },
          },
          "required": [
            "question",
          ],
          "title": "star_wars_qa",
          "type": "object",
          "additionalProperties": false,
        },
        "description": "Multiple-choice question answering",
      },
    },
    {
      "type": "function",
      "function": {
        "name": "person_details",
        "strict": true,
        "parameters": {
          "description": "Identify details about a person",
          "properties": {
            "user": {
              "title": "User",
              "type": "string",
  

In [ ]:
# this should call the person schema tool
introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

response = tool_caller(**{"user": introduction})

print(type(response))
print(format_json(response.model_dump()))

2025-03-06 20:31:24,624 - DEBUG    - yaaal.core.handler - _invoke:183 - Invoking person_details with params: user='Bob'


<class '__main__.Person'>
{
  "name": "Bob",
  "age": 35,
  "favorite_color": "blue",
}


In [ ]:
# this should call the Star Wars QA tool
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = tool_caller(**{"user": question})

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

2025-03-06 20:31:30,248 - DEBUG    - yaaal.core.handler - _invoke:183 - Invoking star_wars_qa with params: question='Han Solo is:\nA. A scoundrel\nB. A scruffy nerfherder\nC. A smuggler\nD. The owner of the Millennium Falcon\nE. All of the above'


<class 'str'>
"E"


In [ ]:
# this should pass through as a normal chat
response = tool_caller(**{"user": "Hello World!"})

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

<class 'str'>
"Hello there! It seems like you've just said a classic programming greeting. Is there anything specific I can help you with today? I have access to some Star Wars-related Q&A and person details tools if you're interested in exploring those."
